In [69]:
import jieba
import collections
import re
from math import *
import matplotlib.pyplot as plt
import numpy
import pandas as pd

jieba.set_dictionary('./dict.txt')

In [31]:
def preprocessing(articles):
    dataset = list()
    for line in articles[:1000]: # 先取少數篇
        sentences = line[:-1].split('t') # print(sentences)
        for i in range(len(sentences)):
            sentences[i] = re.sub(r'\W+', "", sentences[i])
        # print(sentences[0])
        words = list(jieba.cut(sentences[0], cut_all=False))
        dataset.append(words)
    return list(dataset)

In [35]:
def word_tf_in_dataset(dataset):
    all_word_statistics = collections.Counter()
    for article in dataset:
        for sentence in article:
            all_word_statistics += collections.Counter(sentence)
    # print(all_word_statistics)
    total = sum(all_word_statistics.values(), 0.0)
    all_word_tf = {key: round((val/total), 3) for key, val in sorted(all_word_statistics.items(), key=lambda x: x[1], reverse=True)}
    # print(all_word_tf)
    return all_word_tf

In [62]:
def tf(article): # t: 詞
    word_statistics = collections.Counter()
    word_statistics += collections.Counter(list(article))
    total = sum(word_statistics.values(), 0.0)
    word_tf = {key: round((val/total), 2) for key, val in sorted(word_statistics.items(), key=lambda x: x[1], reverse=True)}
    
    return word_tf

In [76]:
def idf(tf_per_article_table): # t: 詞
    word_appear_count = collections.Counter()
    for i in range(len(tf_per_article_table)):
        word_appear_count += collections.Counter({key: 1 for key in tf_per_article_table[i].keys()})
    
    idf_table = {word: round(log10(len(tf_per_article_table) / count), 3) for word, count in sorted(word_appear_count.items(), key=lambda x: x[1], reverse=False)}
    return idf_table

In [78]:
def tf_idf(tf_t_d, idf_t_D):
    table = dict()
    for t, tf in tf_t_d.items():
        if table.get(t) == None:
            table[t] = round((tf * idf_t_D[t]), 3)
        elif table.get(t) != None and round((tf * idf_t_D[t]), 3) > table[t]:
            table[t] = round((tf * idf_t_D[t]), 3)
    return {key: val for key, val in sorted(table.items(), key=lambda x: x[1], reverse=True)}

In [83]:
def rank_100(tf_idf_table):
    skip_flag = {key: False for key in tf_idf_table.keys()}
    rank_100 = {'a': 0.0}
    min_in_rank = ('a', 0.0)
    idx = 0
    while True:
        stop_flag = True
        for i in range(len(tf_idf_table)):
            if idx >= len(tf_idf_table[i].items()): # 第 i 篇文章已經沒人比
                continue # 換下一篇
            if skip_flag[i] != True: # 第 i 篇文章還有詞有競爭力, 不跳過
                word, weight = tuple(tf_idf_table[i].items())[idx] # 把 詞, 權重 用簡單的變數存
                if weight > min_in_rank[1]: # if 權重大於排行榜的最後一名(的權重)
                    rank_100[ word ] = weight # 加進新人
                    if len(rank_100) > 100: # 若排行榜超過 100 人 (還沒滿的話就沒事了)
                        rank_100.pop(min_in_rank[0]) # 踢掉最小
                        new_rank_list = sorted(rank_100.items(), key=lambda x: x[1], reverse=True) # 重排
                        min_in_rank = new_rank_list[-1] # 找出新的最後一名
                        rank_100 = {key: val for key, val in new_rank_list} # 更新 rank_100
                    
                    stop_flag = False
                else:
                    skip_flag[i] = True
        if stop_flag == True:
            break
        idx += 1
    
    return list(rank_100.items())

In [84]:
def main():
    file = open("./hw1-dataset.txt", mode='r', encoding='utf-8')
    articles = file.readlines()
    # print(articles[:3])
    # python 陣列: list
    dataset = preprocessing(articles)
    # print(dataset)
    ### 統計高頻
    tf_whole_dataset_table = word_tf_in_dataset(dataset)
    tf_100 = list(tf_whole_dataset_table.items())[:100]
    # print(tf_100)
    
    ### 建每篇文章的 tf table
    tf_per_article_table = dict()
    for article_no in range(len(dataset)):
        tf_per_article_table[article_no] = tf(dataset[article_no])
        # print(tf_per_article_table[article_no])
    # print(tf_per_article_table)
    # print(pd.DataFrame(list(list(tf_per_article_table.values())[0].items()), columns=['word', 'tf']))
    
    ### 建 idf table
    idf_table = idf(tf_per_article_table)
    # print(idf_table)
    
    ### 統計 tf-idf 高
    tf_idf_table = dict()
    for article_no in range(len(dataset)):
        tf_idf_table[article_no] = tf_idf(tf_per_article_table[article_no], idf_table)
    # print(tf_idf_table)
    tf_idf_100 = rank_100(tf_idf_table)
    # print(tf_idf_100)
    
    file.close()
main()

        
    

[('Come', 3.0), ('廠廠', 2.25), ('S', 1.5), ('dailymo', 1.5), ('P', 1.262), ('林依晨', 0.99), ('培養出', 0.99), ('憂傷', 0.99), ('遺失', 0.99), ('登入', 0.99), ('胖胖', 0.99), ('希拉蕊', 0.99), ('Roo', 0.99), ('MakeAmericaGrea', 0.99), ('黎明', 0.9), ('漫畫', 0.891), ('愛上', 0.891), ('恭喜', 0.891), ('火車站', 0.891), ('加油', 0.891), ('如果', 0.873), ('金剛', 0.87), ('雷電', 0.87), ('醫生', 0.81), ('盡頭', 0.783), ('錢', 0.772), ('感謝', 0.759), ('gay', 0.75), ('Snowbaby', 0.75), ('分手', 0.75), ('咕咕', 0.75), ('瑞雪', 0.75), ('第一個', 0.733), ('p', 0.711), ('姆咪', 0.711), ('高中', 0.681), ('同志', 0.675), ('無法', 0.675), ('核核', 0.66), ('以色列', 0.66), ('雙層', 0.66), ('演', 0.631), ('工具人', 0.631), ('金牌', 0.63), ('台灣', 0.608), ('2016', 0.6), ('通知', 0.6), ('回收', 0.6), ('階段', 0.6), ('國考', 0.6), ('開罐器', 0.6), ('前衝', 0.6), ('張老師', 0.6), ('雞舞', 0.6), ('還開', 0.6), ('帶走', 0.6), ('科學園區', 0.6), ('買裝', 0.6), ('牛舞', 0.6), ('Toyo', 0.6), ('HellKi', 0.6), ('豬舞', 0.6), ('新年', 0.575), ('同性戀', 0.575), ('奮起湖', 0.57), ('被', 0.566), ('甲甲', 0.555), ('you', 0.54), (